<div style="border:solid green 2px; padding: 20px">
    
<b>Привет Команда!</b>

Меня зовут Владислав Струнин и я буду проводить ревью вашего проекта. Моя основная цель — поделиться своим опытом и помочь вам стать отличными специалистами по Data Science. Вами проделана огромная работа над проектом и я предлагаю сделать его еще лучше. Ниже вы найдете мои комментарии - **пожалуйста, не перемещате, не изменяте и не удаляйте их**. Увидев у вас ошибку, я лишь укажу на ее наличие и дам вам возможность самостоятельно найти и исправить ее. <br>
    
Мои комментарии будут выглядеть так:<br>
<div class="alert alert-block alert-success">
<b>✅«Отлично»:</b> Так я выделяю верные действия, когда все сделано правильно.
</div>

<div class="alert alert-warning" role="alert">
<b>⚠️«Можно лучше»: </b> Так выделены небольшие замечания или предложения по улучшению. Я надеюсь, что их вы тоже учтете - проект от этого станет только лучше.
</div>

<div class="alert alert-block alert-danger">
<b>⛔️«Надо исправить»:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

После получения ревью работы постарайтесь внести изменения в исследование в соответствии с моими комментариями. Это позволит сделать вашу работу еще лучше!

In [2]:
import pandas as pd
import numpy as np
from scipy import sparse
from matplotlib import pyplot as plt
from skimpy import clean_columns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier

from sklearn.pipeline import Pipeline, FeatureUnion, FunctionTransformer
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import make_pipeline
from imblearn import FunctionSampler

import pickle

import copy

pd.set_option('display.max_columns', None)


<div class="alert alert-success">
<b>✅«Отлично»:</b><br>
    Хорошо - все импорты собраны в начале тетрадки 👍 Модули, которые относятся к разным фреймворкам можно сгруппировать и разделить пробелом для улучшения восприятия кода.
    
Еще больше интересного есть в руководстве по стилю Python – можно почитать здесь: [PEP 8 – Style Guide for Python Code](https://peps.python.org/pep-0008/)
</div>


In [3]:
# загружаем данные
cargotype_info = pd.read_csv('/home/ivan/projects/Hakaton_Yandex_marketplace/datasets_examples/cargotype_info.csv', index_col=0)
carton_price = pd.read_excel('/home/ivan/projects/Hakaton_Yandex_marketplace/datasets_examples/carton_price.xlsx', names=['cartontype', 'price'])
carton = pd.read_csv('/home/ivan/projects/Hakaton_Yandex_marketplace/datasets_examples/carton.csv')
data = pd.read_csv('/home/ivan/projects/Hakaton_Yandex_marketplace/datasets_examples/data.csv', index_col=0)
sku_cargotypes = pd.read_csv('/home/ivan/projects/Hakaton_Yandex_marketplace/datasets_examples/sku_cargotypes.csv', index_col=0)
sku = pd.read_csv('/home/ivan/projects/Hakaton_Yandex_marketplace/datasets_examples/sku.csv', index_col=0)


<div class="alert alert-success">
<b>✅«Отлично»:</b><br>
    Данные на месте - супер 👍<br>
    Хорошей практикой также является также указывать относительный путь и/или выносить в отдельную переменную (например, DATA_DIR) путь к файлам и тогда, человек, который скопирует ваш проект просто поменяет путь и сможет работать.
</div>


## Предобработка данных

### Сбор единой витрины datamart

In [4]:
# инициализируем datamart
# удаляем ненужные столбцы
datamart = data.drop(['selected_carton', 'recommended_carton', 'whs', 'who', 'trackingid', 'box_num'], axis=1)
# считаем количество товаров одного типа для каждого sku в заказе
datamart = datamart.groupby(list(datamart.columns))['sku'].agg(items_cnt = 'count').reset_index()
# считаем количество уникальных sku в заказе
datamart['sku_cnt'] = datamart.groupby(['orderkey'])['sku'].transform('count')
# считаем количество уникальных типов упаковок в заказе
datamart['cartontypes_cnt'] = datamart.groupby(['orderkey'])['selected_cartontype'].transform(lambda x: len(set(x)))

<div class="alert alert-warning">
<b>⚠️«Можно лучше»: </b><br>
    - cтолбцы, которые вы сразу удаляете после загрузки можно было просто не загружать, если указать параметр <code>usecols</code> в pd.read_csv();<br>
    - человеку, который видит ваш код и тетрадку впервые и не знаком с данными очень поможет, если вы будете показывать результат предобработки после каждого этапа простым вызовом .head().
</div>

In [5]:
# таблица carton
# приводим названия столбцов к змеиному регистру
carton.columns = clean_columns(carton).columns
# удаляем ненужные столбцы
carton = carton.drop(['displayrfpack'], axis=1)

5 column names have been cleaned

In [6]:
# таблица cargotype_info
# удаляем строки с пропущенными значениями
cargotype_info = cargotype_info.dropna()
# форматируем столбцы
cargotype_info['cargotype'] = cargotype_info['cargotype'].astype(int)

In [7]:
# таблица sku_cargotypes
# оставим только те sku, которые есть в data (чтобы быстрее обрабатывать)
sku_cargotypes = sku_cargotypes.query('sku in @data["sku"]')
# сгруппируем по sku, а теги объединим в строку
sku_cargotypes = sku_cargotypes.groupby(['sku'])['cargotype'].apply(list)
sku_cargotypes = sku_cargotypes.apply(lambda x: " ".join(map(str, x)))


In [8]:
# добавляем к datamart линейные размеры товара (datamart left join sku on sku)
datamart = datamart.merge(sku, how='inner', on='sku')
# добавляем к data признаки тегов
datamart = datamart.merge(sku_cargotypes, how='inner', on='sku')

In [9]:
# мерджим тип упаковки с размерами и ценой
carton_data = carton.merge(carton_price, how='left', on='cartontype')

In [10]:
# мерджим инфо по упаковке к datamart
datamart = datamart.merge(carton_data, how='left', left_on='selected_cartontype', right_on='cartontype').drop(['cartontype'], axis=1)
datamart = datamart.merge(carton_data, how='left', left_on='recommended_cartontype', right_on='cartontype', suffixes=('_sel', '_rec')).drop(['cartontype'], axis=1)


In [11]:
# Добавляем фичу объем
datamart['volume'] = datamart['a'] * datamart['b'] * datamart['c']


<div class="alert alert-success">
<b>✅«Отлично»:</b><br>
    Очень уверенно манипулируете данными - подготовили классную общую таблицу, добавили новые фичи 👍
</div>


In [12]:
# определим целевые классы
def target_classif(cartontype):
    # пакеты
    if cartontype in ['MYA', 'MYB', 'MYC', 'MYD', 'MYE']:
        return 1
    # коробки
    elif cartontype in ['YMF', 'YME', 'YMA', 'YMW', 'YMG', 'YML', 'YMC', 'MYF', 'YMX', 'YMB']:
        return 2
    # без упаковки или стретч
    elif cartontype in ['NONPACK', 'STRETCH']:
        return 0
    else:
        return 0


# зададим целевые классы
datamart['y_target'] = datamart['selected_cartontype'].apply(target_classif)
datamart['y_old_model'] = datamart['recommended_cartontype'].apply(target_classif)


<div class="alert alert-block alert-danger">
<b>⛔️«Надо исправить»:</b><br>
    Вы решаете очень обобщенную задачу, которая не соответствует задаче, сформулированной Заказчиком в <a href = "https://prairie-parade-285.notion.site/95e473fb67e54aefa6e1c26cd7a8b0e1">Техническом задании</a>: уменьшить время, которое Пользователь тратит на подбор упаковочного материала. Не думаю, что выбор пакет или коробка занимает много времени, скорее выбор <b>какой</b> пакет или <b>какая</b> коробка. Необходимо увеличить число классов и рекомендовать тару с учетом весогабаритных харакетристик товара.<br>
    Однако, можно не отказываться от построенной вами модели и использовать ее для предварительной классификации заказов по обобщенному признаку - пакет/коробка/пленка и далее передавать классифицированный заказ спеицализированным моделям, которые отвечают на вопросы: <b>какой</b> пакет или <b>какая</b> коробка:)
</div>

In [13]:
# обновим таблицу с данными об упаковках для задачи оптимизации
carton_data['y_target'] = carton_data['cartontype'].apply(target_classif)
carton_data = carton_data[pd.isna(carton_data['y_target']) == False]
carton_data = carton_data.fillna(0)

In [13]:
# запишем данные об упаковках в отдельный csv файл
# carton_data.to_csv('/home/ivan/projects/Hakaton_Yandex_marketplace/carton_data.csv')


In [14]:
# посмотрим на f1_score старой модели
f1_score(datamart['y_old_model'], datamart['y_target'], average='macro')

0.5392833241583701


<div class="alert alert-success">
<b>✅«Отлично»:</b><br>
    Не хватает пояснений, но попробую догадаться - вы так определили baseline? 🧙 Окей, посмотрим получится ли его улучшить:)  Еще можно добавить матрицу ошибок и посмотреть где модель ошибается чаще.
</div>


In [15]:
# запакуем единую витрину в csv
# datamart.head(100).to_csv('/home/ivan/projects/Hakaton_Yandex_marketplace/datasets_examples/datamart_example.csv')

## Моделирование

In [15]:
# готовим данные для модели
# model_data = datamart[['orderkey', 'sku', 'items_cnt', 'a', 'b', 'c', 'volume', 'goods_wght', 'cargotype', 'y_target']]
model_data = datamart[['orderkey', 'sku', 'items_cnt', 'a', 'b', 'c', 'goods_wght', 'cargotype', 'y_target']]
# составим список уникальных заказов
unique_orderkeys = pd.Series(model_data['orderkey'].unique())
# поделим заказы на train, valid и test 60/20/20
train_orderkeys, test_orderkeys = train_test_split(unique_orderkeys, test_size=0.2, random_state=123)


# зададим функцию разделения данных на X и y
def X_y_split(model_data, orders, y):
    model_data_orders = model_data.query('orderkey in @orders')
    model_data_orders_X = model_data_orders.drop([y, 'orderkey', 'sku'], axis=1).reset_index(drop=True)
    model_data_orders_y = model_data_orders[y].reset_index(drop=True)
    return model_data_orders_X, model_data_orders_y


# используем функцию разделения данных
train_X, train_y = X_y_split(model_data=model_data, orders=train_orderkeys, y='y_target')
test_X, test_y = X_y_split(model_data=model_data, orders=test_orderkeys, y='y_target')


<div class="alert alert-block alert-danger">
<b>⛔️«Надо исправить»:</b><br>
    Как вы считаете, после дропа orderkey и sku - сколько дубликатов будет в наборе данных? Думаю много, значит - здесь может быть утечка данных: в тест могут попасть такие же данные как в трейне. Это нужно обязательно проконтролировать и предупредить.
</div>

<div class="alert alert-warning">
<b>⚠️«Можно лучше»: </b><br>
    Вижу по закомментированным строкам, что экспериментировали с тем, какие признаки подавать в модель для обучения. Снова повторю, что ход своих мыслей лучше показывать☝️ Также добавлю, что выводы о том, какие признаки полезные, а какие нет, лучше делать опираясь на их интерпретацию с помощью значений feature_importance или инструментов бибилиотеки sklearn - <a href = "https://scikit-learn.org/stable/modules/feature_selection.html">Feature Selection</a>.
</div>

<div class="alert alert-warning">
<b>⚠️«Можно лучше»: </b><br>
    В таблице после группировки по orderkey + sku каждая строка стала аналогом отдельного товара в заказе и вы соответственно предсказываете не набор упаковки для заказа, а упаковку для отдельного товара. Этап EDA отсутствует в вашей работе и если человек не знаком с данными, то он посчитает, что вы сильно упрощаете задачу. В базе 97% заказов состоит из одного SKU и вы можете побороться за оставшиеся 3%, если придумаете как построить признаки таким образом, чтобы строка ассоциировалась с заказом. Варианты как это сделать обсуждались на совместной встрече:)
</div>

In [17]:
# # сохраним примеры данных для входа в модель
# model_in = model_data.drop(['y_target'], axis=1).head(1000)
# model_in.to_csv('/home/ivan/projects/Hakaton_Yandex_marketplace/project_data/data_in.csv')

In [18]:
# # сохраним примеры данных на выходе из модели (смоделированные)
# model_out = copy.deepcopy(model_in)
# model_out['class_0'] = np.random.choice([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], model_out.shape[0])
# model_out['class_1'] = np.random.choice([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], model_out.shape[0])
# model_out['class_2'] = np.random.choice([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], model_out.shape[0])

# model_out['class_0'] = model_out['class_0'] / (model_out['class_0'] + model_out['class_1'] + model_out['class_2'])
# model_out['class_1'] = model_out['class_1'] / (model_out['class_0'] + model_out['class_1'] + model_out['class_2'])
# model_out['class_2'] = model_out['class_2'] / (model_out['class_0'] + model_out['class_1'] + model_out['class_2'])

# model_out.to_csv('/home/ivan/projects/Hakaton_Yandex_marketplace/project_data/data_out.csv')

In [16]:
# Создадим pipeline
pipeline = make_pipeline()
# векторизуем признак cargotype 
vectorizer = ColumnTransformer([("vectorizer", TfidfVectorizer(min_df=10, max_df=1.0), 'cargotype')],remainder="passthrough")
pipeline.steps.append(('vectorizer', vectorizer))
# добавим классификатор
pipeline.steps.append(('clf', KNeighborsClassifier(n_jobs=-1, n_neighbors=15)))


<div class="alert alert-success">
<b>✅«Отлично»:</b><br>
    Интересное решение - векторизовать карготип:) Ограничили частоту - тоже хорошо, тем самым (возможно?) уменьшили словарь на выходе - это стоит проверить. <br>
    Также хороший вариант - создать бинарный признак для каждого уникального значения cargotype, получится максимум 88 новых признаков на выходе. Это можно сделать с помощью pivot таблиц или использовать BOW вместо TF-IDF.
</div>


<div class="alert alert-warning">
<b>⚠️«Можно лучше»: </b><br>
    Параметр n_jobs=-1 можно также указать для ColumnTransformer и тогда он будет работать быстрее и использовать ресурсы эффективнее.
</div>

In [17]:
# формат данных на въод модели
train_X.head()

,items_cnt,a,b,c,goods_wght,cargotype
0,1,17.0,38.0,6.0,1.00,290 410 750 780
1,2,17.0,38.0,6.0,1.00,290 410 750 780
2,3,25.0,7.0,17.0,0.09,290 340 410 720 750 780
3,2,25.0,7.0,17.0,0.09,290 340 410 720 750 780
4,2,25.0,7.0,17.0,0.09,290 340 410 720 750 780


In [18]:
# обучим модель
pipeline.fit(train_X, train_y)

Pipeline(steps=[('vectorizer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('vectorizer',
                                                  TfidfVectorizer(min_df=10),
                                                  'cargotype')])),
                ('clf', KNeighborsClassifier(n_jobs=-1, n_neighbors=15))])

In [19]:
# сохраним модель
import joblib
joblib.dump(pipeline, "knn_model.joblib")

['knn_model.joblib']

In [23]:
# посмотрим на f1_score новой модели
f1_score(pipeline.predict(test_X), test_y, average='macro')

0.6747875981635253


<div class="alert alert-success">
<b>✅«Отлично»:</b><br>
    Baseline побит! Поздравляю! Время помучить гиперпараметры: счастье не в соседях, а в их количестве:) А может и попробовать другие модели?
</div>


<div style="border:solid Chocolate 2px; padding: 40px">
Спасибо за ваш проект! Получилось интересно и есть над чем поработать:)

<b>Положительные моменты проекта</b>:
    <ol>
        <li>Хорошо и уверенно работаете с несколькими таблицами, подготовили хорошую основную таблицу;</li>
        <li>Интересное решение с векторизацией карготипов;</li>
        <li>Проверили адекватность модели с помощью baseline.</li>
    </ol>

<b>Что необходимо исправить</b>:
    <ol>
        <li>Не выполнен этап EDA, очень мало пояснений и нет никакой визуализации - все таки это исследование;)</li>
        <li>Задачу классификации слишком обобщили и тем самым упростили - требования ТЗ не выполнены;</li>
        <li>Данные для обучения и тестирования нужно проверить на предмет утечек из-за дублей;</li>
        <li>Для ревью не предоставлен Docker-контейнер.</li>
    </ol>   
    
<b>На что еще стоит обратить внимание</b>:
    <ol>
        <li>Пожалейте того, кто не знаком с данными и видит вашу работу впервые - показывайте с чем работаете;</li>
        <li>Этап предобработки лучше оформить в Pipeline.</li>
    </ol>

    Желаю удачи и побед в соревнованиях!😉
</div>